In [1]:
import uproot
import numpy as np
import VisualisationFileHelper

In [6]:
#fileName = '/Users/isobel/Desktop/DUNE/2024/Hierarchy/files/nu_dune10kt_1x2x6_1421_369_20230827T093357Z_gen_g4_detsim_hitreco_ccnutree.root'
#fileName = '/Users/isobel/Desktop/DUNE/2024/Hierarchy/files/nue_dune10kt_1x2x6_1124_526_20230828T185125Z_gen_g4_detsim_hitreco_ccnutree.root'
fileName = '/Users/isobel/Desktop/DUNE/2024/Hierarchy/files/ccnutree.root'

In [7]:
treeFile = uproot.open(fileName)
tree = treeFile['ccnuselection/ccnusel']
branches = tree.arrays()

In [8]:
# Get relevant branches...
pfpTrueVisibleGeneration = branches['RecoPFPTrueVisibleGeneration']
pfpTrueGeneration = branches['RecoPFPTrueGeneration']
pfpRecoGeneration = branches['RecoPFPRecoGeneration']
pfpRecoVertexX = branches['RecoPFPRecoVertexX']
pfpTruePDG = branches['RecoPFPTruePDG']
pfpTrueTrackID = branches['RecoPFPTrueTrackID']
pfpTrueVisibleParentPDG = branches['RecoPFPTrueVisibleParentPDG']
pfpTrueVisibleParentTrackID = branches['RecoPFPTrueVisibleParentTrackID']
pfpRecoParentSelf = branches['RecoPFPRecoParentSelf']
pfpSelf = branches['RecoPFPSelf']

isNC = branches['NC']
nuPDG = branches['NuPdg']


run = branches['Run']
subrun = branches['SubRun']
event = branches['Event']

In [9]:
# Process files creating lists that can be turned into numpy arrays for easier processing

# Event arrays
# -------------------
nRecoPrimary = []
nRecoSecondary = []
nRecoTertiary = []

nRecoPrimaryCorrect = []
nRecoSecondaryCorrect = []
nRecoTertiaryCorrect = []

nTrueVisiblePrimary = []
nTrueVisibleSecondary = []
nTrueVisibleTertiary = []

nEvents = len(pfpRecoGeneration)

# Particle counts
# -------------------
nRecoPrimaryCorrect_CC_muon = 0
nTrueVisiblePrimary_CC_muon = 0

nRecoPrimaryCorrect_CC_electron = 0
nTrueVisiblePrimary_CC_electron = 0

nRecoPrimaryCorrect_decay_proton = 0
nTrueVisiblePrimary_decay_proton = 0

nRecoPrimaryCorrect_michel_electron = 0
nTrueVisiblePrimary_michel_electron = 0

# Tier counts
# -------------------
nTrueVisOne = 0
nTrueVisOneRecoOne = 0
nTrueVisOneRecoTwo = 0
nTrueVisOneRecoThree = 0

nTrueVisTwo = 0
nTrueVisTwoRecoOne = 0
nTrueVisTwoRecoTwo = 0
nTrueVisTwoRecoThree = 0

nTrueVisThree = 0
nTrueVisThreeRecoOne = 0
nTrueVisThreeRecoTwo = 0
nTrueVisThreeRecoThree = 0

nTrueVisFour = 0
nTrueVisFourRecoOne = 0
nTrueVisFourRecoTwo = 0
nTrueVisFourRecoThree = 0
nTrueVisFourRecoFour = 0

for iEvent in range(nEvents) :
    
    # Get np arrays for particles in the event
    recoGeneration = np.array(pfpRecoGeneration[iEvent])
    trueVisibleGeneration = np.array(pfpTrueVisibleGeneration[iEvent])
    trueGeneration = np.array(pfpTrueGeneration[iEvent])
    truePDG = np.array(pfpTruePDG[iEvent])
    
    # Make sure that they have a 3D representation, this isn't the best way to do it
    recoGeneration = recoGeneration[pfpRecoVertexX[iEvent] > -999]
    trueVisibleGeneration = trueVisibleGeneration[pfpRecoVertexX[iEvent] > -999]
    trueGeneration = trueGeneration[pfpRecoVertexX[iEvent] > -999]
    truePDG = truePDG[pfpRecoVertexX[iEvent] > -999]    
    
    # Note no. true generations    
    nTrueVisiblePrimary.append(np.count_nonzero(trueVisibleGeneration == 2))
    nTrueVisibleSecondary.append(np.count_nonzero(trueVisibleGeneration == 3))
    nTrueVisibleTertiary.append(np.count_nonzero(trueVisibleGeneration == 4))
    
    # Note no. reco generations 
    nRecoPrimary.append(np.count_nonzero(recoGeneration == 2))
    nRecoSecondary.append(np.count_nonzero(recoGeneration == 3))
    nRecoTertiary.append(np.count_nonzero(recoGeneration == 4))
    
    # Note no. correctly reco'd particles
    correctReco = (recoGeneration == trueVisibleGeneration) # [true, false, false, ...]
    nRecoPrimaryCorrect.append(np.count_nonzero(np.logical_and(correctReco, trueVisibleGeneration == 2)))
    nRecoSecondaryCorrect.append(np.count_nonzero(np.logical_and(correctReco, trueVisibleGeneration == 3)))
    nRecoTertiaryCorrect.append(np.count_nonzero(np.logical_and(correctReco, trueVisibleGeneration == 4)))

    # Note no. CC muons
    if ((isNC[iEvent] == 0) and (abs(nuPDG[iEvent]) == 14)) : 
        nTrueVisiblePrimary_CC_muon += np.count_nonzero(np.logical_and(np.fabs(truePDG) == 13, trueVisibleGeneration == 2))
        nRecoPrimaryCorrect_CC_muon += np.count_nonzero(np.logical_and(np.fabs(truePDG) == 13, np.logical_and(correctReco, trueVisibleGeneration == 2)))
    
    # Note no. electrons
    if ((isNC[iEvent] == 0) and (abs(nuPDG[iEvent]) == 12)) : 
        nTrueVisiblePrimary_CC_electron += np.count_nonzero(np.logical_and(np.fabs(truePDG) == 11, trueVisibleGeneration == 2))
        nRecoPrimaryCorrect_CC_electron += np.count_nonzero(np.logical_and(np.fabs(truePDG) == 11, np.logical_and(correctReco, trueVisibleGeneration == 2)))
    
    # Note no. 'decay/reinteraction' protons
    nTrueVisiblePrimary_decay_proton += np.count_nonzero(np.logical_and(np.fabs(truePDG) == 2212, np.logical_and(trueVisibleGeneration == 2, trueGeneration != 2)))
    nRecoPrimaryCorrect_decay_proton += np.count_nonzero(np.logical_and(np.fabs(truePDG) == 2212, np.logical_and(correctReco, np.logical_and(trueVisibleGeneration == 2, trueGeneration != 2))))

    # Note tier spread..
    nTrueVisOne += np.count_nonzero(trueVisibleGeneration == 2)
    nTrueVisOneRecoOne += np.count_nonzero(np.logical_and(trueVisibleGeneration == 2, recoGeneration == 2))
    nTrueVisOneRecoTwo += np.count_nonzero(np.logical_and(trueVisibleGeneration == 2, recoGeneration == 3))
    nTrueVisOneRecoThree += np.count_nonzero(np.logical_and(trueVisibleGeneration == 2, recoGeneration == 4))
    
    nTrueVisTwo += np.count_nonzero(trueVisibleGeneration == 3)
    nTrueVisTwoRecoOne += np.count_nonzero(np.logical_and(trueVisibleGeneration == 3, recoGeneration == 2))
    nTrueVisTwoRecoTwo += np.count_nonzero(np.logical_and(trueVisibleGeneration == 3, recoGeneration == 3))
    nTrueVisTwoRecoThree += np.count_nonzero(np.logical_and(trueVisibleGeneration == 3, recoGeneration == 4))
    
    nTrueVisThree += np.count_nonzero(trueVisibleGeneration == 4)
    nTrueVisThreeRecoOne += np.count_nonzero(np.logical_and(trueVisibleGeneration == 4, recoGeneration == 2))
    nTrueVisThreeRecoTwo += np.count_nonzero(np.logical_and(trueVisibleGeneration == 4, recoGeneration == 3))
    nTrueVisThreeRecoThree += np.count_nonzero(np.logical_and(trueVisibleGeneration == 4, recoGeneration == 4))

    nTrueVisFour += np.count_nonzero(trueVisibleGeneration == 5)
    nTrueVisFourRecoOne += np.count_nonzero(np.logical_and(trueVisibleGeneration == 5, recoGeneration == 2))
    nTrueVisFourRecoTwo += np.count_nonzero(np.logical_and(trueVisibleGeneration == 5, recoGeneration == 3))
    nTrueVisFourRecoThree += np.count_nonzero(np.logical_and(trueVisibleGeneration == 5, recoGeneration == 4))
    nTrueVisFourRecoFour += np.count_nonzero(np.logical_and(trueVisibleGeneration == 5, recoGeneration == 5))
    
# turn into numpy arrays
nRecoPrimary = np.array(nRecoPrimary)
nRecoSecondary = np.array(nRecoSecondary)
nRecoTertiary = np.array(nRecoTertiary)

nRecoPrimaryCorrect = np.array(nRecoPrimaryCorrect)
nRecoSecondaryCorrect = np.array(nRecoSecondaryCorrect)
nRecoTertiaryCorrect = np.array(nRecoTertiaryCorrect)

nTrueVisiblePrimary = np.array(nTrueVisiblePrimary)
nTrueVisibleSecondary = np.array(nTrueVisibleSecondary)
nTrueVisibleTertiary = np.array(nTrueVisibleTertiary)

In [10]:
# Look at delta rays + michels...
# -----------------------------------------------

nTrueMichels = 0
correctMichelParents = 0

# Get self of true delta-ray/michel reco parents
michelParentMuonSelf_v = pfpRecoParentSelf[np.logical_and(np.abs(pfpTruePDG) == 11, np.abs(pfpTrueVisibleParentPDG) == 13)]

for iEvent in range(len(michelParentMuonSelf_v)) :
    
    ##################################
    #if (len(michelParentMuonSelf_v[iEvent]) != 0) :
        #print('run:', run[iEvent])
        #print('subrun:', subrun[iEvent])
        #print('event:', event[iEvent])
    ##################################
    
    for iParticle in range(len(michelParentMuonSelf_v[iEvent])) : 
        
        nTrueMichels += 1
        
        # The self of the reco parent
        michelParentMuonSelf = michelParentMuonSelf_v[iEvent][iParticle]
        
        # Find the index so we can work out the reco parent matched trackID
        # This gives (vector of all times mentioned, first index mentioned)
        # If it is not found, it's because the self is of the reco neutrino 
        parentMuonIndex = np.where(np.array(pfpSelf[iEvent]) == michelParentMuonSelf)
        
        # If mentioned, see if parent is actually our muon
        if (len(parentMuonIndex[0]) != 0) :
            
            trueRecoParentTrackID = pfpTrueTrackID[iEvent][parentMuonIndex[0][0]]
            
            if (trueRecoParentTrackID == pfpTrueVisibleParentTrackID[iEvent][iParticle]) :
                correctMichelParents += 1

In [11]:
# Look at proton -> proton links
# -----------------------------------------------

nTrueProtonProtonLinks = 0
correctProtonProtonLinks = 0

# Get self of true proton reco parents
targetParentSelf_v = pfpRecoParentSelf[np.logical_and(np.abs(pfpTruePDG) == 2212, np.abs(pfpTrueVisibleParentPDG) == 2212)]

for iEvent in range(len(targetParentSelf_v)) :
    
    ##################################
    if (len(targetParentSelf_v[iEvent]) != 0) :
        print('run:', run[iEvent])
        print('subrun:', subrun[iEvent])
        print('event:', event[iEvent])
    ##################################
    
    
    for iParticle in range(len(targetParentSelf_v[iEvent])) : 
        
        nTrueProtonProtonLinks += 1
        
        # The self of the reco parent
        targetParentSelf = targetParentSelf_v[iEvent][iParticle]
        
        # Find the index so we can work out the reco parent matched trackID
        # This gives (vector of all times mentioned, first index mentioned)
        # If it is not found, it's because the self is of the reco neutrino 
        targetParentIndex = np.where(np.array(pfpSelf[iEvent]) == targetParentSelf)
        
        # If mentioned, see if parent is actually our muon
        if (len(targetParentIndex[0]) != 0) :
            
            trueRecoParentTrackID = pfpTrueTrackID[iEvent][targetParentIndex[0][0]]
            
            if (trueRecoParentTrackID == pfpTrueVisibleParentTrackID[iEvent][iParticle]) :
                correctProtonProtonLinks += 1
                

run: 1111
subrun: 1
event: 8302
run: 1111
subrun: 1
event: 8309
run: 1111
subrun: 1
event: 8314
run: 1111
subrun: 1
event: 8328
run: 1111
subrun: 1
event: 8331
run: 1111
subrun: 1
event: 8345
run: 1111
subrun: 1
event: 8346
run: 1111
subrun: 1
event: 8353
run: 1111
subrun: 1
event: 8354
run: 1111
subrun: 1
event: 8372
run: 1111
subrun: 1
event: 8373
run: 1111
subrun: 1
event: 8375
run: 1111
subrun: 1
event: 8397
run: 1111
subrun: 1
event: 8398
run: 1111
subrun: 1
event: 8400
run: 1111
subrun: 1
event: 8501
run: 1111
subrun: 1
event: 8507
run: 1111
subrun: 1
event: 8514
run: 1111
subrun: 1
event: 8522
run: 1111
subrun: 1
event: 8525
run: 1111
subrun: 1
event: 8533
run: 1111
subrun: 1
event: 8534
run: 1111
subrun: 1
event: 8547
run: 1111
subrun: 1
event: 8557
run: 1111
subrun: 1
event: 8562
run: 1111
subrun: 1
event: 8585
run: 1111
subrun: 1
event: 86520
run: 1111
subrun: 1
event: 86524
run: 1111
subrun: 1
event: 86525
run: 1111
subrun: 1
event: 86538
run: 1111
subrun: 1
event: 86546
run

run: 1412
subrun: 1
event: 40749
run: 1412
subrun: 1
event: 40753
run: 1412
subrun: 1
event: 40772
run: 1412
subrun: 1
event: 40796
run: 1412
subrun: 1
event: 41603
run: 1412
subrun: 1
event: 41611
run: 1412
subrun: 1
event: 41623
run: 1412
subrun: 1
event: 41644
run: 1412
subrun: 1
event: 41649
run: 1412
subrun: 1
event: 41658
run: 1412
subrun: 1
event: 41667
run: 1412
subrun: 1
event: 41674
run: 1412
subrun: 1
event: 41689
run: 1412
subrun: 1
event: 41691
run: 1412
subrun: 1
event: 42603
run: 1412
subrun: 1
event: 42608
run: 1412
subrun: 1
event: 42621
run: 1412
subrun: 1
event: 42625
run: 1412
subrun: 1
event: 42631
run: 1412
subrun: 1
event: 42634
run: 1412
subrun: 1
event: 42635
run: 1412
subrun: 1
event: 42645
run: 1412
subrun: 1
event: 42656
run: 1412
subrun: 1
event: 42658
run: 1412
subrun: 1
event: 42678
run: 1412
subrun: 1
event: 42688
run: 1412
subrun: 1
event: 42696
run: 1412
subrun: 1
event: 43502
run: 1412
subrun: 1
event: 43504
run: 1412
subrun: 1
event: 43527
run: 1412


run: 1106
subrun: 1
event: 95356
run: 1106
subrun: 1
event: 95373
run: 1106
subrun: 1
event: 95381
run: 1106
subrun: 1
event: 95388
run: 1106
subrun: 1
event: 95396
run: 1409
subrun: 1
event: 39123
run: 1409
subrun: 1
event: 39125
run: 1409
subrun: 1
event: 39126
run: 1409
subrun: 1
event: 39133
run: 1409
subrun: 1
event: 39139
run: 1409
subrun: 1
event: 39140
run: 1409
subrun: 1
event: 39171
run: 1409
subrun: 1
event: 39178
run: 1409
subrun: 1
event: 39184
run: 1409
subrun: 1
event: 81012
run: 1409
subrun: 1
event: 81020
run: 1409
subrun: 1
event: 81025
run: 1409
subrun: 1
event: 81037
run: 1409
subrun: 1
event: 81054
run: 1409
subrun: 1
event: 81071
run: 1409
subrun: 1
event: 81072
run: 1409
subrun: 1
event: 81077
run: 1409
subrun: 1
event: 81090
run: 1409
subrun: 1
event: 81091
run: 1411
subrun: 1
event: 24509
run: 1411
subrun: 1
event: 24510
run: 1411
subrun: 1
event: 24525
run: 1411
subrun: 1
event: 24528
run: 1411
subrun: 1
event: 24535
run: 1411
subrun: 1
event: 24538
run: 1411


run: 1418
subrun: 1
event: 40755
run: 1418
subrun: 1
event: 40757
run: 1418
subrun: 1
event: 40772
run: 1418
subrun: 1
event: 40775
run: 1418
subrun: 1
event: 40782
run: 1418
subrun: 1
event: 40792
run: 1418
subrun: 1
event: 49401
run: 1418
subrun: 1
event: 49408
run: 1418
subrun: 1
event: 49411
run: 1418
subrun: 1
event: 49414
run: 1418
subrun: 1
event: 49420
run: 1418
subrun: 1
event: 49428
run: 1418
subrun: 1
event: 49439
run: 1418
subrun: 1
event: 49452
run: 1418
subrun: 1
event: 49475
run: 1418
subrun: 1
event: 53310
run: 1418
subrun: 1
event: 53335
run: 1418
subrun: 1
event: 53336
run: 1418
subrun: 1
event: 53343
run: 1418
subrun: 1
event: 53364
run: 1418
subrun: 1
event: 53385
run: 1418
subrun: 1
event: 6009
run: 1418
subrun: 1
event: 6013
run: 1418
subrun: 1
event: 6039
run: 1418
subrun: 1
event: 6044
run: 1418
subrun: 1
event: 6048
run: 1418
subrun: 1
event: 6049
run: 1418
subrun: 1
event: 6080
run: 1418
subrun: 1
event: 6102
run: 1418
subrun: 1
event: 6104
run: 1418
subrun: 1

In [12]:
# Look at broken muons
# -----------------------------------------------

nTrueBrokenMuons = 0
connectedMuons = 0

# Get self of true proton reco parents
muonMask = np.logical_and(pfpRecoVertexX > -999, np.logical_and(np.abs(pfpTruePDG) == 13, pfpTrueGeneration == 2))

for iEvent in range(len(muonMask)) : 
        
    if ((isNC[iEvent] == 1) or (abs(nuPDG[iEvent]) != 14)) :      
        continue
        
    muonTrackIDs = pfpTrueTrackID[muonMask][iEvent]
    muonRecoSelf = pfpSelf[muonMask][iEvent]
    muonRecoParentSelf = pfpRecoParentSelf[muonMask][iEvent]
    
    if (len(muonTrackIDs) != 2) :
        continue
        
    nTrueBrokenMuons += 1
        
    connected1 = muonRecoSelf[0] == muonRecoParentSelf[1]
    connected2 = muonRecoSelf[1] == muonRecoParentSelf[0]
        
    if (connected1 or connected2) :
        connectedMuons += 1

In [13]:
##############################################
# Do tier spread  metrics
##############################################
trueVisOneRecoOneFraction = float(nTrueVisOneRecoOne) / float(nTrueVisOne) if nTrueVisOne != 0 else -1
trueVisOneRecoTwoFraction = float(nTrueVisOneRecoTwo) / float(nTrueVisOne) if nTrueVisOne != 0 else -1
trueVisOneRecoThreeFraction = float(nTrueVisOneRecoThree) / float(nTrueVisOne) if nTrueVisOne != 0 else -1

trueVisTwoRecoOneFraction = float(nTrueVisTwoRecoOne) / float(nTrueVisTwo) if nTrueVisTwo != 0 else -1
trueVisTwoRecoTwoFraction = float(nTrueVisTwoRecoTwo) / float(nTrueVisTwo) if nTrueVisTwo != 0 else -1
trueVisTwoRecoThreeFraction = float(nTrueVisTwoRecoThree) / float(nTrueVisTwo) if nTrueVisTwo != 0 else -1

trueVisThreeRecoOneFraction = float(nTrueVisThreeRecoOne) / float(nTrueVisThree) if nTrueVisThree != 0 else -1
trueVisThreeRecoTwoFraction = float(nTrueVisThreeRecoTwo) / float(nTrueVisThree) if nTrueVisThree != 0 else -1
trueVisThreeRecoThreeFraction = float(nTrueVisThreeRecoThree) / float(nTrueVisThree) if nTrueVisThree != 0 else -1

trueVisFourRecoOneFraction = float(nTrueVisFourRecoOne) / float(nTrueVisFour) if nTrueVisFour != 0 else -1
trueVisFourRecoTwoFraction = float(nTrueVisFourRecoTwo) / float(nTrueVisFour) if nTrueVisFour != 0 else -1
trueVisFourRecoThreeFraction = float(nTrueVisFourRecoThree) / float(nTrueVisFour) if nTrueVisFour != 0 else -1
trueVisFourRecoFourFraction = float(nTrueVisFourRecoFour) / float(nTrueVisFour) if nTrueVisFour != 0 else -1

In [14]:
print('------------------------------------------------------')
print('nTrueVisOne:', nTrueVisOne)
print('trueVisOneRecoOneFraction:', str(round(trueVisOneRecoOneFraction * 100.0, 2)) + '%')
print('trueVisOneRecoTwoFraction:', str(round(trueVisOneRecoTwoFraction * 100.0, 2)) + '%')
print('trueVisOneRecoThreeFraction:', str(round(trueVisOneRecoThreeFraction * 100.0, 2)) + '%')
print('------------------------------------------------------')
print('nTrueVisTwo:', nTrueVisTwo)
print('trueVisTwoRecoOneFraction:', str(round(trueVisTwoRecoOneFraction * 100.0, 2)) + '%')
print('trueVisTwoRecoTwoFraction:', str(round(trueVisTwoRecoTwoFraction * 100.0, 2)) + '%')
print('trueVisTwoRecoThreeFraction:', str(round(trueVisTwoRecoThreeFraction * 100.0, 2)) + '%')
print('------------------------------------------------------')
print('nTrueVisThree:', nTrueVisThree)
print('trueVisThreeRecoOneFraction:', str(round(trueVisThreeRecoOneFraction * 100.0, 2)) + '%')
print('trueVisThreeRecoTwoFraction:', str(round(trueVisThreeRecoTwoFraction * 100.0, 2)) + '%')
print('trueVisThreeRecoThreeFraction:', str(round(trueVisThreeRecoThreeFraction * 100.0, 2)) + '%')
print('------------------------------------------------------')
print('nTrueVisFour:', nTrueVisFour)
print('trueVisFourRecoOneFraction:', str(round(trueVisFourRecoOneFraction * 100.0, 2)) + '%')
print('trueVisFourRecoTwoFraction:', str(round(trueVisFourRecoTwoFraction * 100.0, 2)) + '%')
print('trueVisFourRecoThreeFraction:', str(round(trueVisFourRecoThreeFraction * 100.0, 2)) + '%')
print('trueVisFourRecoFourFraction:', str(round(trueVisFourRecoFourFraction * 100.0, 2)) + '%')

------------------------------------------------------
nTrueVisOne: 95896
trueVisOneRecoOneFraction: 96.59%
trueVisOneRecoTwoFraction: 3.22%
trueVisOneRecoThreeFraction: 0.16%
------------------------------------------------------
nTrueVisTwo: 18341
trueVisTwoRecoOneFraction: 61.32%
trueVisTwoRecoTwoFraction: 36.92%
trueVisTwoRecoThreeFraction: 1.6%
------------------------------------------------------
nTrueVisThree: 2338
trueVisThreeRecoOneFraction: 63.34%
trueVisThreeRecoTwoFraction: 5.73%
trueVisThreeRecoThreeFraction: 29.13%
------------------------------------------------------
nTrueVisFour: 221
trueVisFourRecoOneFraction: 71.04%
trueVisFourRecoTwoFraction: 0.45%
trueVisFourRecoThreeFraction: 5.88%
trueVisFourRecoFourFraction: 21.27%


In [15]:
##############################################
# Do particle level metrics
##############################################

correctlyTieredFraction_CC_muon = float(nRecoPrimaryCorrect_CC_muon) / float (nTrueVisiblePrimary_CC_muon) if nTrueVisiblePrimary_CC_muon != 0 else -1
correctlyTieredFraction_CC_electron = float(nRecoPrimaryCorrect_CC_electron) / float (nTrueVisiblePrimary_CC_electron) if nTrueVisiblePrimary_CC_electron != 0 else -1
correctlyTieredFraction_decay_proton = float(nRecoPrimaryCorrect_decay_proton) / float (nTrueVisiblePrimary_decay_proton) if nTrueVisiblePrimary_decay_proton != 0 else -1
correctMuonShowerLinkFraction = float(correctMichelParents) / float(nTrueMichels) if nTrueMichels != 0 else -1
correctProtonProtonLinkFraction = float(correctProtonProtonLinks) / float(nTrueProtonProtonLinks) if nTrueProtonProtonLinks != 0 else -1
connectedMuonsFraction = float(connectedMuons) / float(nTrueBrokenMuons) if nTrueBrokenMuons else -1


In [16]:
print('------------------------------------------------------')
print('nTrueVisiblePrimary_CC_muon:', nTrueVisiblePrimary_CC_muon)
print('nRecoPrimaryCorrect_CC_muon:', nRecoPrimaryCorrect_CC_muon)
print('correctlyTieredFraction_CC_muon:', str(round(correctlyTieredFraction_CC_muon * 100.0, 2)) + '%')
print('------------------------------------------------------')
print('nTrueVisiblePrimary_CC_electron:', nTrueVisiblePrimary_CC_electron)
print('nRecoPrimaryCorrect_CC_electron:', nRecoPrimaryCorrect_CC_electron)
print('correctlyTieredFraction_CC_electron:', str(round(correctlyTieredFraction_CC_electron * 100.0, 2)) + '%')
print('------------------------------------------------------')
print('nTrueVisiblePrimary_decay_proton:', nTrueVisiblePrimary_decay_proton)
print('nRecoPrimaryCorrect_decay_proton:', nRecoPrimaryCorrect_decay_proton)
print('correctlyTieredFraction_decay_proton:', str(round(correctlyTieredFraction_decay_proton * 100.0, 2)) + '%')
print('------------------------------------------------------')
print('nTrueMichels:', nTrueMichels)
print('correctMichelParents:', correctMichelParents)
print('correctMuonShowerLinkFraction:', str(round(correctMuonShowerLinkFraction * 100.0, 2)) + '%')
print('------------------------------------------------------')
print('nTrueProtonProtonLinks:', nTrueProtonProtonLinks)
print('correctProtonProtonLinks:', correctProtonProtonLinks)
print('correctProtonProtonLinkFraction:', str(round(correctProtonProtonLinkFraction * 100.0, 2)) + '%')
print('------------------------------------------------------')
print('nTrueBrokenMuons', nTrueBrokenMuons)
print('connectedMuons', connectedMuons)
print('connectedMuonsFraction:', str(round(connectedMuonsFraction * 100.0, 2)) + '%')

------------------------------------------------------
nTrueVisiblePrimary_CC_muon: 13595
nRecoPrimaryCorrect_CC_muon: 12174
correctlyTieredFraction_CC_muon: 89.55%
------------------------------------------------------
nTrueVisiblePrimary_CC_electron: 9753
nRecoPrimaryCorrect_CC_electron: 9526
correctlyTieredFraction_CC_electron: 97.67%
------------------------------------------------------
nTrueVisiblePrimary_decay_proton: 10702
nRecoPrimaryCorrect_decay_proton: 10446
correctlyTieredFraction_decay_proton: 97.61%
------------------------------------------------------
nTrueMichels: 584
correctMichelParents: 1
correctMuonShowerLinkFraction: 0.17%
------------------------------------------------------
nTrueProtonProtonLinks: 3905
correctProtonProtonLinks: 117
correctProtonProtonLinkFraction: 3.0%
------------------------------------------------------
nTrueBrokenMuons 1631
connectedMuons 514
connectedMuonsFraction: 31.51%


In [17]:
##############################################
# Work out two types of metrics
# 1. If the tier isn't empty, what does the reco look like
##############################################

# First do correct tier fraction
# ---------------------------------

# Get the no. particle vectors of the true, non-empty tiers
nTrueVisiblePrimary_notEmpty = nTrueVisiblePrimary[nTrueVisiblePrimary != 0]
nTrueVisibleSecondary_notEmpty = nTrueVisibleSecondary[nTrueVisibleSecondary != 0]
nTrueVisibleTertiary_notEmpty = nTrueVisibleTertiary[nTrueVisibleTertiary != 0]

nRecoPrimary_notEmpty = nRecoPrimary[nTrueVisiblePrimary != 0]
nRecoSecondary_notEmpty = nRecoSecondary[nTrueVisibleSecondary != 0]
nRecoTertiary_notEmpty = nRecoTertiary[nTrueVisibleTertiary != 0]

nRecoPrimaryCorrect_notEmpty = nRecoPrimaryCorrect[nTrueVisiblePrimary != 0]
nRecoSecondaryCorrect_notEmpty = nRecoSecondaryCorrect[nTrueVisibleSecondary != 0]
nRecoTertiaryCorrect_notEmpty = nRecoTertiaryCorrect[nTrueVisibleTertiary != 0]

# Is the tier correct?
# i.e. all reco particles there and no others...
primaryTierCorrect_notEmpty = np.logical_and((nRecoPrimaryCorrect_notEmpty == nTrueVisiblePrimary_notEmpty), (nRecoPrimary_notEmpty == nTrueVisiblePrimary_notEmpty))
secondaryTierCorrect_notEmpty = np.logical_and((nRecoSecondaryCorrect_notEmpty == nTrueVisibleSecondary_notEmpty), (nRecoSecondary_notEmpty == nTrueVisibleSecondary_notEmpty))
tertiaryTierCorrect_notEmpty = np.logical_and((nRecoTertiaryCorrect_notEmpty == nTrueVisibleTertiary_notEmpty), (nRecoTertiary_notEmpty == nTrueVisibleTertiary_notEmpty))

# Correct Event fractions
primaryTierCorrectFraction_notEmpty = round(float(np.count_nonzero(primaryTierCorrect_notEmpty)) / float(primaryTierCorrect_notEmpty.shape[0]) * 100.0, 2)
secondaryTierCorrectFraction_notEmpty = round(float(np.count_nonzero(secondaryTierCorrect_notEmpty)) / float(secondaryTierCorrect_notEmpty.shape[0]) * 100.0, 2)
tertiaryTierCorrectFraction_notEmpty = round(float(np.count_nonzero(tertiaryTierCorrect_notEmpty)) / float(tertiaryTierCorrect_notEmpty.shape[0]) * 100.0, 2)


print('-------')
print('Primary: ')
print('-------')
#print('nTrueVisiblePrimary:', nTrueVisiblePrimary_notEmpty)
#print('nRecoPrimary:', nRecoPrimary_notEmpty)
#print('nRecoPrimaryCorrect:', nRecoPrimaryCorrect_notEmpty)
print('primaryTierCorrectFraction:', primaryTierCorrectFraction_notEmpty, '%')

print('-------')
print('Secondary: ')
print('-------')
#print('nTrueVisibleSecondary:', nTrueVisibleSecondary_notEmpty)
#print('nRecoSecondary:', nRecoSecondary_notEmpty)
#print('nRecoSecondaryCorrect:', nRecoSecondaryCorrect_notEmpty)
print('secondaryTierCorrectFraction:', secondaryTierCorrectFraction_notEmpty, '%')

print('-------')
print('Tertiary: ')
print('-------')
#print('nTrueVisibleTertiary:', nTrueVisibleTertiary_notEmpty)
#print('nRecoTertiary:', nRecoTertiary_notEmpty)
#print('nRecoTertiaryCorrect:', nRecoTertiaryCorrect_notEmpty)
print('tertiaryTierCorrectFraction:', tertiaryTierCorrectFraction_notEmpty, '%')


-------
Primary: 
-------
primaryTierCorrectFraction: 68.27 %
-------
Secondary: 
-------
secondaryTierCorrectFraction: 30.56 %
-------
Tertiary: 
-------
tertiaryTierCorrectFraction: 28.93 %


In [18]:
# Next to do completeness/purity
# ---------------------------------

# Need to turn things into floats, so I can divide...

nRecoPrimary_notEmpty = nRecoPrimary_notEmpty.astype(float)
nRecoSecondary_notEmpty = nRecoSecondary_notEmpty.astype(float)
nRecoTertiary_notEmpty = nRecoTertiary_notEmpty.astype(float)

nRecoPrimaryCorrect_notEmpty = nRecoPrimaryCorrect_notEmpty.astype(float)
nRecoSecondaryCorrect_notEmpty = nRecoSecondaryCorrect_notEmpty.astype(float)
nRecoTertiaryCorrect_notEmpty = nRecoTertiaryCorrect_notEmpty.astype(float)

nTrueVisiblePrimary_notEmpty = nTrueVisiblePrimary_notEmpty.astype(float)
nTrueVisibleSecondary_notEmpty = nTrueVisibleSecondary_notEmpty.astype(float)
nTrueVisibleTertiary_notEmpty = nTrueVisibleTertiary_notEmpty.astype(float)

# Calculate completeness, shouldn't be any zeros here..

completenessPrimary_notEmpty = np.divide(nRecoPrimaryCorrect_notEmpty, nTrueVisiblePrimary_notEmpty) * 100.0
completenessSecondary_notEmpty = np.divide(nRecoSecondaryCorrect_notEmpty, nTrueVisibleSecondary_notEmpty) * 100.0
completenessTertiary_notEmpty = np.divide(nRecoTertiaryCorrect_notEmpty, nTrueVisibleTertiary_notEmpty) * 100.0

avCompletenessPrimary_notEmpty = np.average(completenessPrimary_notEmpty)
avCompletenessSecondary_notEmpty = np.average(completenessSecondary_notEmpty)
avCompletenessTertiary_notEmpty = np.average(completenessTertiary_notEmpty)

print('avCompletenessPrimary:', round(avCompletenessPrimary_notEmpty, 2), '%')
print('avCompletenessSecondary:', round(avCompletenessSecondary_notEmpty, 2), '%')
print('avCompletenessTertiary:', round(avCompletenessTertiary_notEmpty, 2), '%')     

# Calculate purity, shouldn't be any zeros here..

purityPrimary_notEmpty = np.divide(nRecoPrimaryCorrect_notEmpty, nRecoPrimary_notEmpty, out=np.zeros_like(nRecoPrimaryCorrect_notEmpty), where=nRecoPrimary_notEmpty!=0)
puritySecondary_notEmpty = np.divide(nRecoSecondaryCorrect_notEmpty, nRecoSecondary_notEmpty, out=np.zeros_like(nRecoSecondaryCorrect_notEmpty), where=nRecoSecondary_notEmpty!=0)
purityTertiary_notEmpty= np.divide(nRecoTertiaryCorrect_notEmpty, nRecoTertiary_notEmpty, out=np.zeros_like(nRecoTertiaryCorrect_notEmpty), where=nRecoTertiary_notEmpty!=0)

avPurityPrimary_notEmpty = np.average(purityPrimary_notEmpty)
avPuritySecondary_notEmpty = np.average(puritySecondary_notEmpty)
avPurityTertiary_notEmpty = np.average(purityTertiary_notEmpty)

print('----------------------------------------')
print('purityPrimary:', round(avPurityPrimary_notEmpty, 2), '%')
print('puritySecondary:', round(avPuritySecondary_notEmpty, 2), '%')
print('purityTertiary:', round(avPurityTertiary_notEmpty, 2), '%')                                                                                                                                                                                         
                                                                                                            

avCompletenessPrimary: 97.35 %
avCompletenessSecondary: 44.54 %
avCompletenessTertiary: 34.33 %
----------------------------------------
purityPrimary: 0.93 %
puritySecondary: 0.53 %
purityTertiary: 0.38 %
